In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-18"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11434,2024-10-18,Brasil Nbb,19:30,Fortaleza B.C.,União Corinthians,1.22,3.68,151.5,1.83,1.83,-10.5,1.95,1.72,SOoom5zI,0.819672,0.271739,0.546448,0.546448,0.091411,0.2,0.8,NaN,NaN,128.594,24.549991,0.190911,223.820,62.306732,0.278379,104.734,364.370,89.68,295.36,0.0,0.0,0.0,0.0,0.709993,0.000000,0.088629,0.48,0.096,2.291667,0.665210,0.7,0.034790,-2.80,-0.560,-4.785714,0.273053,0.3,0.026947
11435,2024-10-18,Brasil Nbb,19:30,Unifacisa,Caxias do Sul,1.15,4.51,150.5,1.82,1.84,-12.5,1.95,1.72,MRrwkR55,0.869565,0.221729,0.549451,0.543478,0.091295,0.0,0.6,NaN,NaN,117.422,19.828414,0.168865,190.734,74.101466,0.388507,106.026,272.890,100.43,276.08,0.0,0.0,0.0,0.0,0.839533,0.007728,0.088629,-1.25,-0.250,-0.600000,0.740494,0.8,0.059506,-5.00,-1.000,-3.510000,0.310984,0.2,-0.110984
11436,2024-10-18,Europa Euroliga,14:00,Monaco,Virtus Bologna,1.26,4.14,157.5,1.91,1.91,-9.5,2.00,1.71,lG1J2nlR,0.793651,0.241546,0.523560,0.523560,0.035197,0.4,0.6,NaN,NaN,120.118,37.676316,0.313661,177.582,78.270545,0.440757,104.934,194.384,86.10,162.35,0.0,0.0,0.0,0.0,0.754247,0.000000,0.110545,0.04,0.008,32.500000,0.732964,0.7,-0.032964,-0.33,-0.066,-47.575758,0.434041,0.2,-0.234041
11437,2024-10-18,Europa Euroliga,14:30,Anadolu Efes,Olympiakos,2.05,1.80,157.5,1.91,1.91,1.5,1.91,1.91,4t7lcQBl,0.487805,0.555556,0.523560,0.523560,0.043360,0.4,0.2,NaN,NaN,140.156,52.438120,0.374141,140.778,40.201632,0.285568,134.936,160.792,140.40,200.64,0.0,0.0,0.0,0.0,0.091832,0.000000,0.000000,-0.26,-0.052,-20.192308,0.637736,0.7,0.062264,-0.73,-0.146,-5.479452,0.438647,0.4,-0.038647
11438,2024-10-18,Europa Euroliga,15:00,Crvena zvezda,Barcelona,1.87,1.95,156.5,1.91,1.91,-1.5,1.95,1.87,Qeece4t1,0.534759,0.512821,0.523560,0.523560,0.047580,0.4,0.6,NaN,NaN,179.854,102.241029,0.568467,134.780,57.885115,0.429479,147.946,175.068,93.73,107.87,0.0,0.0,0.0,0.0,0.029617,0.000000,0.029617,-0.41,-0.082,-10.609756,0.681090,0.6,-0.081090,-0.64,-0.128,-7.421875,0.503096,0.4,-0.103096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11516,2024-10-18,República Tcheca 1. Liga,15:00,Hradec Kralove,Novy Jicin,1.83,1.83,162.5,1.83,1.83,-28.5,1.83,1.83,EqjoIFdC,0.546448,0.546448,0.546448,0.546448,0.092896,0.0,0.6,NaN,NaN,982.586,1211.438326,1.232908,260.572,332.529502,1.276152,0.000,692.922,107.10,854.70,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,22.63,4.526,0.183385,0.000000,0.0,0.000000
11517,2024-10-18,Rússia Superliga,11:00,Chelyabinsk,CSKA Moscow 2,1.56,2.32,144.5,1.85,1.85,-4.5,1.95,1.72,8rTXrPMm,0.641026,0.431034,0.540541,0.540541,0.072060,0.2,0.6,NaN,NaN,145.362,47.308324,0.325452,136.570,34.323995,0.251329,109.224,147.742,84.24,114.00,0.0,0.0,0.0,0.0,0.277011,0.000000,0.088629,2.30,0.460,1.217391,0.000000,0.0,0.000000,-1.11,-0.222,-5.945946,0.000000,0.0,0.000000
11518,2024-10-18,Rússia Superliga,11:00,Metallurg,Tambov,1.47,2.56,143.5,1.85,1.85,-6.5,1.95,1.72,G8Wus3i0,0.680272,0.390625,0.540541,0.540541,0.070897,0.0,0.8,NaN,NaN,NaN,NaN,NaN,229.154,101.632071,0.443510,0.000,267.894,161.20,399.57,0.0,0.0,0.0,0.0,0.382504,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,-0.87,-0.174,-8.965517,0.000000,0.0,0.000000
11519,2024-10-18,Rússia Premier League Feminina,09:00,Enisey Krasnoyarsk F,Neftyanik Omsk F,1.41,2.76,130

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:30,Europa Euroliga,Anadolu Efes,Olympiakos,157.5,1.91,0.8383,Over
1,15:00,Europa Euroliga,Crvena zvezda,Barcelona,156.5,1.91,0.7478,Over
2,15:30,Europa Euroliga,Lyon-Villeurbanne,Baskonia,165.5,1.91,0.7262,Over
